In [56]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Input, Dropout, Conv2D, Flatten
from sklearn.preprocessing import MinMaxScaler as MMS, StandardScaler as SDS
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
import pandas as pd

# 1. 데이터

In [57]:
path = 'C:/study/keras/keras_data/ddarung/'
# path = 'C:/study/keras_data/ddarung/'

train_data = pd.read_csv(path + 'train.csv', index_col = 0) # index_col = 0 → id 열 데이터로 취급 X
test_data = pd.read_csv(path + 'test.csv', index_col = 0)
submission = pd.read_csv(path + 'submission.csv', index_col = 0)

# ---------------------- 결측치 처리 (제거)----------------------- #
train_data = train_data.dropna()

# ---------------------- x,y 분리 ------------------------ #
x = train_data.drop(['count'], axis=1)  # y 값(count 열) 분리, axis = 1 → 열에 대해 동작
y = train_data['count']                 # y 값(count 열)만 추출

# print(train_data.shape)        # (1459, 10)
# print(test_data.shape)         # (715, 9)
# print(train_data.columns)      'hour', 'hour_bef_temperature', 'hour_bef_precipitation',
#                               'hour_bef_windspeed', 'hour_bef_humidity', 'hour_bef_visibility',
#                               'hour_bef_ozone', 'hour_bef_pm10', 'hour_bef_pm2.5', 'count'
# print(train_data.info())       # Missing Attribute Values: 결측치 - 데이터에 값이 없는 것
# print(train_data.describe())   # 평균, 표준편차, 최대값 등

x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.7, random_state=44)

scaler = MMS()
# scaler = SDS()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

print("x_train: ", x_train.shape, "x_test:", x_test.shape)

# # ---------- CNN 모델에 적용해보기 위해 4차원으로 변환 ----------- #
x_train = x_train.reshape(929, 9, 1, 1)
x_test = x_test.reshape(399, 9, 1, 1)


x_train:  (929, 9) x_test: (399, 9)


# 2. 모델


In [58]:
model = Sequential()
model.add(Conv2D(32, (2,1), input_shape = (9, 1, 1), activation='relu'))
model.add(Dropout(0.5)) # 과적합 방지
model.add(Conv2D(16, (2,1), activation='relu'))
model.add(Dropout(0.3)) # 과적합 방지
model.add(Flatten())    # DNN모델에 적용하기 위해 2차원으로 변환
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2)) # 과적합 방지
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3)) # 과적합 방지
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5)) # 과적합 방지
model.add(Dense(128, activation='relu'))
model.add(Dense(1))

# 3. 컴파일 및 훈련

In [59]:
model.compile(loss='mse', optimizer='adam') # one-hot encoding 하지 않아도 되는 데이터이므로 loss= sparse_categorical_crossentropy

ES = EarlyStopping(monitor='val_loss', mode='auto', patience=4, restore_best_weights=True) 
model.fit(x_train, y_train, epochs=64, batch_size=8, validation_split=0.2, callbacks = [ES], verbose=2) # verbose: 함수 수행시 발생하는 상세한 정보들을 표준 출력으로 자세히 내보낼 것인지

Epoch 1/64


c:\Users\bitcamp\anaconda3\envs\tf27\lib\site-packages\keras\engine\data_adapter.py:1510: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  return t[start:end]


93/93 - 1s - loss: 9259.2090 - val_loss: 4522.1533 - 610ms/epoch - 7ms/step
Epoch 2/64
93/93 - 0s - loss: 5600.7085 - val_loss: 3534.1721 - 171ms/epoch - 2ms/step
Epoch 3/64
93/93 - 0s - loss: 4898.2656 - val_loss: 3030.9338 - 165ms/epoch - 2ms/step
Epoch 4/64
93/93 - 0s - loss: 4438.1313 - val_loss: 2619.7874 - 162ms/epoch - 2ms/step
Epoch 5/64
93/93 - 0s - loss: 4344.7520 - val_loss: 2641.5234 - 155ms/epoch - 2ms/step
Epoch 6/64
93/93 - 0s - loss: 4122.8110 - val_loss: 2425.1387 - 164ms/epoch - 2ms/step
Epoch 7/64
93/93 - 0s - loss: 3976.3660 - val_loss: 2617.4070 - 160ms/epoch - 2ms/step
Epoch 8/64
93/93 - 0s - loss: 4208.7817 - val_loss: 2331.4126 - 158ms/epoch - 2ms/step
Epoch 9/64
93/93 - 0s - loss: 4236.7319 - val_loss: 2318.7617 - 157ms/epoch - 2ms/step
Epoch 10/64
93/93 - 0s - loss: 3865.8660 - val_loss: 2259.4800 - 158ms/epoch - 2ms/step
Epoch 11/64
93/93 - 0s - loss: 3541.0977 - val_loss: 2268.5786 - 161ms/epoch - 2ms/step
Epoch 12/64
93/93 - 0s - loss: 3779.1646 - val_loss:

# 4. 평가 및 예측

In [60]:
loss = model.evaluate(x_test, y_test, verbose=2)
print('loss(mse): ', loss)

y_predict = model.predict(x_test)

RMSE = np.sqrt(mean_squared_error(y_test, y_predict))
print("RMSE: ", RMSE)

r2 = r2_score(y_test, y_predict)
print("R2: ", r2)

13/13 - 0s - loss: 2738.4016 - 23ms/epoch - 2ms/step
loss(mse):  2738.401611328125
RMSE:  52.32974004517006
R2:  0.5968960353565047
